In [31]:
import pandas as pd
import json
import urllib.request
import math
import sys
import requests
from pprint import pprint

In [34]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [39]:
# Terms:
#  Dollar Volume Liquidity:  is a stock's share price multiplied by its daily share volume.

# Liquidity vs. Volume Trading volume is simply a measure of the value of executed trades within a period of time, typically measured on a daily basis. Liquidity, on the other hand, has more to do with the buy and sell orders that are currently on the order books. In other words, volume is a measurement of the trades that have already taken place, while liquidity informs the buy and sell offers that can currently be accepted on the exchange.

In [24]:
# Links
# https://cryptomarketpool.com/use-the-graph-to-query-ethereum-data-in-python/
# https://hasura.io/docs/latest/queries/postgres/sorting/
# https://www.marketcalls.in/trading-lessons/understand-basic-difference-liquidity-volume.html
# https://www.cryptovantage.com/guides/what-does-liquidity-mean-for-cryptocurrency/

In [11]:
# URL = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"
url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

In [15]:
# Convert Uniswap v3 tick to a price (i.e. the ratio between the amounts of tokens: token1/token0)
def tick_to_price(tick):
    return TICK_BASE ** tick

# Not all ticks can be initialized. Tick spacing is determined by the pool's fee tier.
def fee_tier_to_tick_spacing(fee_tier):
    return {
        100: 1,
        500: 10,
        3000: 60,
        10000: 200
    }.get(fee_tier, 60)
# function to use requests.post to make an API call to the subgraph url
def run_query(q):

    # endpoint where you are making the request
    request = requests.post(url,
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))


In [ ]:
# Top 20 tokens by TVL



In [18]:
# Get top N pools by totalValueLockedUSD

N = 50
pool_ids = {}
#query pools($top_n: ID!){
# Following link teaches how to query using graphql
# https://hasura.io/docs/latest/queries/postgres/sorting/

query = """
{
    pools(
        first: $top_n
        orderBy:totalValueLockedUSD
        orderDirection: desc
      ) {
        id
        createdAtTimestamp
        tick
        token0 {
          symbol
          name
          decimals
          totalSupply
          totalValueLocked
          totalValueLockedUSD
          volumeUSD
          feesUSD
        }
        token1 {
          symbol
          name
          decimals
          totalSupply
          totalValueLocked
          totalValueLockedUSD
          volumeUSD
          feesUSD
        }
        collectedFeesUSD
        collectedFeesToken0
        collectedFeesToken1
        feeTier
        feesUSD
        liquidity
        liquidityProviderCount
        token0Price
        token1Price
        totalValueLockedUSD
        totalValueLockedToken0
        totalValueLockedToken1
        volumeToken0
        volumeToken1
        volumeUSD

  } 
}
"""

# Query the subgraph
# req = urllib.request.Request(URL)
# req.add_header('Content-Type', 'application/json; charset=utf-8')
# jsondata = {"query": query, "variables": {"top_n": N}}
# jsondataasbytes = json.dumps(jsondata).encode('utf-8')
# req.add_header('Content-Length', len(jsondataasbytes))
# response = urllib.request.urlopen(req, jsondataasbytes)
# obj = json.load(response)
# print(obj)

result = run_query(query)

In [25]:

# print the results
print('Print Result - {}'.format(result))
print('#############')

Print Result - {'data': {'pools': [{'id': '0x277667eb3e34f134adf870be9550e9f323d0dc24', 'createdAtTimestamp': '1663808675', 'tick': '-161389', 'token0': {'symbol': 'ease.org', 'name': 'Ease Fun Token', 'decimals': '18', 'totalSupply': '28368', 'totalValueLocked': '1003900.001085209227447215', 'totalValueLockedUSD': '1109778722632.18178148433647518817', 'volume': '0.000003', 'volumeUSD': '1.141758918984956706118360126972235', 'feesUSD': '0.0001141758918984956706118360126972235'}, 'token1': {'symbol': 'ez-cvxsteCRV', 'name': 'cvxsteCRV Ease Vault', 'decimals': '18', 'totalSupply': '18368', 'totalValueLocked': '0.098157689878957356', 'totalValueLockedUSD': '0', 'volume': '0.000000000000108977', 'volumeUSD': '0', 'feesUSD': '0'}, 'feeTier': '100', 'feesUSD': '0', 'liquidity': '313521001529099829406', 'liquidityProviderCount': '0', 'token0Price': '10201983.52821130596040573863212092', 'token1Price': '0.00000009802015433907763482013950488687576', 'totalValueLockedUSD': '1110167010695.7859000

In [26]:
pprint(result)

{'data': {'pools': [{'createdAtTimestamp': '1663808675',
                     'feeTier': '100',
                     'feesUSD': '0',
                     'id': '0x277667eb3e34f134adf870be9550e9f323d0dc24',
                     'liquidity': '313521001529099829406',
                     'liquidityProviderCount': '0',
                     'tick': '-161389',
                     'token0': {'decimals': '18',
                                'feesUSD': '0.0001141758918984956706118360126972235',
                                'name': 'Ease Fun Token',
                                'symbol': 'ease.org',
                                'totalSupply': '28368',
                                'totalValueLocked': '1003900.001085209227447215',
                                'totalValueLockedUSD': '1109778722632.18178148433647518817',
                                'volume': '0.000003',
                                'volumeUSD': '1.141758918984956706118360126972235'},
                     'tok

In [29]:
pools = result['data']['pools']
token_cols = pools[0]['token0'].keys()
for idx,p in enumerate(pools):
    for x in ['token0', 'token1']:
        for c in token_cols:
            pools[idx][f'{x}_{c}'] = p[x][c]

In [36]:
df1 = pd.DataFrame(pools)
df1.drop(columns=['token0', 'token1'],inplace = True)

In [38]:
# Double checking fee tiers
df1.feeTier.value_counts()
# 0.01%, 0.05%, 0.3%, 1%

3000     41
10000    31
500      19
100       9
Name: feeTier, dtype: int64

In [37]:
df1.head(20)

,id,createdAtTimestamp,tick,feeTier,feesUSD,liquidity,liquidityProviderCount,token0Price,token1Price,totalValueLockedUSD,totalValueLockedToken0,totalValueLockedToken1,token0_symbol,token0_name,token0_decimals,token0_totalSupply,token0_totalValueLocked,token0_totalValueLockedUSD,token0_volume,token0_volumeUSD,token0_feesUSD,token1_symbol,token1_name,token1_decimals,token1_totalSupply,token1_totalValueLocked,token1_totalValueLockedUSD,token1_volume,token1_volumeUSD,token1_feesUSD
0,0x277667eb3e34f134adf870be9550e9f323d0dc24,1663808675,-161389,100,0,313521001529099829406,0,10201983.52821130596040573863212092,0.00000009802015433907763482013950488687576,1110167010695.785900004139211677005,1001400.000100999992665055,0.098157689878957355,ease.org,Ease Fun Token,18,28368,1003900.001085209227447215,1109778722632.18178148433647518817,0.000003,1.141758918984956706118360126972235,0.0001141758918984956706118360126972235,ez-cvxsteCRV,cvxsteCRV Ease Vault,18,18368,0.098157689878957356,0,0.000000000000108977,0,0
1,0xa850478adaace4c08fc61de44d8cf3b64f359bec,1625360210,46048,500,0,1706245281880037395956227425,0,0.01000586374960270227477201921193648,99.94139686738253752770898237370869,12887625156.60443191907094774602519,50050480.103497613002904654,14644676667.240849617715772909,UMIIE,UMIIE COIN,18,28240,4060050480.115629263002904667,1418175887838.43006967093349508692,1000111.36015031931450555,2234.275824537063717565341059307139,6.271907994682834462724163690131096,UMIIE2,Umiie 2,18,18240,14644676667.240849617715772909,0,99930988.660196871689931931,0,0
2,0x8c0411f2ad5470a66cb2e9c64536cfb8dcd54d51,1663808063,-69281,100,0,40740981393984479118,0,1020.197309047320743679301892429615,0.0009802025462445285537950420182116627,1432077483.208630196467118329099649,1300.000100909928897755,1.275526246490417916,ease.org,Ease Fun Token,18,28368,1003900.001085209227447215,1109778722632.18178148433647518817,0.000003,1.141758918984956706118360126972235,0.0001141758918984956706118360126972235,ez-yvCurve-IronBank,yvCurve-IronBank Ease Vault,18,18368,1.275526246490417916,0,0.000000000989903616,0,0
3,0x055284a4ca6532ecc219ac06b577d540c686669d,1663808363,None,100,0,0,0,0,0,1322031013.848042060988788389158996,1200.000883299205884404,0.000000000011998846,ease.org,Ease Fun Token,18,28368,1003900.001085209227447215,1109778722632.18178148433647518817,0.000003,1.141758918984956706118360126972235,0.0001141758918984956706118360126972235,ez-SLP-WBTC-WETH,WBTC-WETH SLP Ease Vault,18,18368,0.000000000011998846,0,0,0,0
4,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1620250931,201945,500,176335369.7312392274134553224813291,27824787316974085166,0,1698.490446887185584214725250244457,0.0005887580950677083204895498392741363,363895234.8212010928449184232377324,206504407.168735,92677.860456415387571803,USDC,USD Coin,6,19312,801395568.310042,801395568.310042,578002542397.742837,577854594925.0263620360667153467773,532114681.817181633237908037807917,WETH,Wrapped Ether,18,19848,619211.810301398934441546,1051580807.277610276529651291639711,333052396.331927194386288771,734150905329.2055584971525209729411,1183210897.988011021105904017405557
5,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1620169800,201946,3000,208019721.5949497116981142984034039,8558213679362796850,0,1698.25702575950129012860472691456,0.0005888390183769597572338250018119705,301810368.8427692551257789555622563,164281858.755152,80982.152878838352243477,USDC,USD Coin,6,19312,801395568.310042,801395568.310042,578002542397.742837,577854594925.0263620360667153467773,532114681.817181633237908037807917,WETH,Wrapped Ether,18,19848,619211.810301398934441546,1051580807.277610276529651291639711,333052396.331927194386288771,734150905329.2055584971525209729411,1183210897.988011021105904017405557
6,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,1620158974,257048,3000,62153916.4779745001952768470312026,2476971529823551261,0,0.06872243815054344389454108624163311,14.55128815146806882165860160254078,269035278.0462043314533046672687245,3901.09805404,100555.0

In [20]:
# Look at the USDC/ETH 0.3% pool
POOL_ID = '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'


TICK_BASE = 1.0001

# GraphQL query to get the pool information
query = """query pools($pool_id: ID!) {
  pools (where: {id: $pool_id}) {
    tick
    liquidity
    feeTier
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""



# Query the subgraph
req = urllib.request.Request(URL)
req.add_header('Content-Type', 'application/json; charset=utf-8')
jsondata = {"query": query, "variables": {"pool_id": POOL_ID}}
jsondataasbytes = json.dumps(jsondata).encode('utf-8')
req.add_header('Content-Length', len(jsondataasbytes))
response = urllib.request.urlopen(req, jsondataasbytes)
obj = json.load(response)
print(obj)
pool = obj['data']['pools'][0]

# Extract liquidity from the response
L = int(pool["liquidity"])
tick = int(pool["tick"])
tick_spacing = fee_tier_to_tick_spacing(int(pool["feeTier"]))

print("L={}".format(L))
print("tick={}".format(tick))

token0 = pool["token0"]["symbol"]
token1 = pool["token1"]["symbol"]
decimals0 = int(pool["token0"]["decimals"]) # USDC has 6 decimals
decimals1 = int(pool["token1"]["decimals"]) # WETH has 18 decimals

# Compute the tick range. This code would work as well in Python: `tick // TICK_SPACING * TICK_SPACING`
# However, using floor() is more portable.
bottom_tick = math.floor(tick / tick_spacing) * tick_spacing
top_tick = bottom_tick + tick_spacing

# Compute the current price and adjust it to a human-readable format
price = tick_to_price(tick)
adjusted_price = price / (10 ** (decimals1 - decimals0))

# Compute square roots of prices corresponding to the bottom and top ticks
sa = tick_to_price(bottom_tick // 2)
sb = tick_to_price(top_tick // 2)
sp = price ** 0.5

# Compute real amounts of the two assets
amount0 = L * (sb - sp) / (sp * sb)
amount1 = L * (sp - sa)

# Adjust them to a human-readable format
adjusted_amount0 = amount0 / 10 ** decimals0
adjusted_amount1 = amount1 / 10 ** decimals1

print("Current price: {:.6f} {} for 1 {} ({:.6f} {} for 1 {})".format(
    adjusted_price, token1, token0, 1 / adjusted_price, token0, token1))

print("Amounts at the current tick range: {:.2f} {} and {:.2f} {}".format(
    adjusted_amount0, token0, adjusted_amount1, token1))

{'data': {'pools': [{'tick': '201946', 'liquidity': '8558213679362796850', 'feeTier': '3000', 'token0': {'symbol': 'USDC', 'decimals': '6'}, 'token1': {'symbol': 'WETH', 'decimals': '18'}}]}}
L=8558213679362796850
tick=201946
Current price: 0.000589 WETH for 1 USDC (1698.381094 USDC for 1 WETH)
Amounts at the current tick range: 246788.57 USDC and 477.06 WETH
